# Daily Summary of EPA Data Sets
### Data Engineering Capstone Project

<font color='red'>Note: To run this notebook an AWS key pair is required with S3 and Redshift read/write access as well as an existing Redshift cluster. The notebook pulls the information from the file `dwh.cfg`. The files in S3 will also need to be deleted after completion. <BR>**NO QUOTES in `dwh.cfg`**</font>

#### Project Summary
The project will gather several data sets from the EPA website and aggregate their entries for time series analysis, The ETL for this project will involve downloading, processing with Pandas, uploading to AWS S3, and loading and joining with AWS Redshift.

<img src="images/pipeline.png" alt="data pipeline" style="width:80%">

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
import os
import sys
import configparser

import requests
import psycopg2
import boto3

import numpy as np
import pandas as pd
import pandas.io.sql as sqlio

### Step 1: Scope the Project and Gather Data

#### Scope 
The project will transform several hourly data sets from the EPA website into daily aggregated data suitable for time series analysis. The hourly data will be aggregated by state and day with summary statistics using Pandas and exported to CSV. The CSV files will be uploaded to an S3 bucket and `COPY`-ed into a Redshift database. The tables will then be `JOIN`-ed into tables containing the maximums and averages for each state and day.

#### Describe and Gather Data 
The EPA website has daily, hourly, and 8-hour data for pollutants and meteorologic data aggregated by year. The data is contained in zipped CSV files with similar download URLs, varying by the EPA code and year, making it easy to programmatically download the data files. The ETL workflow will work for any of the hourly files over the course of years from 1980 to present:

<img src="images/epa_page-criteria gases.png" alt="epa_page-criteria gases" style="width:50%">
<img src="images/epa_page-meteorological.png" alt="epa_page-meteorological" style="width:50%">

The default setup for this notebook will be download Ozone, SO2, CO, NO2, and Temperature for 2018 (the latest full year).

In [ ]:
code_dict = {'Ozone': 44201,
             'SO2': 42401,
             'CO': 42101,
             'NO2': 42602,
             'Temperature': 'TEMP'}

In [ ]:
start = 2018
end = 2018
years = range(start, end+1)

More information about the files and the data in the CSV tables can be found at EPA website:
    https://aqs.epa.gov/aqsweb/airdata/FileFormats.html
An excerpt of that webpage specific to the hourly data files can be found in `AirData Download Files Documentation.pdf`.

##### Specify Directory for Data

In [ ]:
data_folder_name = 'data'

##### Download Data

In [ ]:
def create_directory(folder_name):
    # check that save directory exists, otherwise make directory
    print(f"checking for directory {folder_name}")
    if not os.path.exists(folder_name):
        print("directory for data doesn't exist")
        print(f"creating directory at {folder_name}\n")
        os.makedirs(folder_name)
    else:
        print("data directory exists\n")

def get_file_from_url(url, save_location=""):
    # check that save directory exists, otherwise make directory
    create_directory(save_location)
    # check for file name from url, otherwise download programatically
    file_name = url.split('/')[-1]
    file_path = os.path.join(save_location, file_name)
    if not os.path.isfile(file_path):
        print(f"file {file_name} doesn't exist at {save_location}")
        print(f"downloading {file_name} to {file_path}\n")
        with open(file_path, mode = 'wb') as file:
            file.write(requests.get(url).content)
    else:
        print(f'file {file_name} already exists at {file_path}\n')

In [ ]:
# create data directory
create_directory(data_folder_name) 

# programatically download data files
#
# urls for hourly downloads are in the form:
#    https://aqs.epa.gov/aqsweb/airdata/hourly_{CODE}_{YEAR}.zip
# where code corresponds to a number or abbreviation.
#
# years imported above
# years is a `range` of years
# years = range(start, end+1)
#
# code_dict imported above
# code_dict is a dictionary corresponding to names and EPA codes
# example:
#   code_dict = {'Temperature (62101)': 'TEMP',
#                 'Ozone': 44201,
#                 'SO2': 42401,
#                 'CO': 42101,
#                 'NO2': 42602}

for title, code in code_dict.items():
    for year in years:
        working_dir = os.path.join(data_folder_name, title)
        file_name = f'hourly_{code}_{year}.zip'
        get_file_from_url(f'https://aqs.epa.gov/aqsweb/airdata/hourly_{code}_{year}.zip',
                          working_dir)

### Step 2: Explore and Assess the Data
#### Explore the Data & Cleaning Steps
Example data is explored in `data_exploration.ipynb`. The files examined are `hourly_TEMP_2016.zip` and `hourly_42101_2016.zip`, which are temperature and carbon monoxide data from 2016. The takeaways from examining both files are that sites record the exact longitude and latitude, which are diferent for every reading. There are different sites or multiple readings for each kind of data set, so the bucket size needs to be bigger to compare time series data. The bucket sizes chosen were grouping by state and using daily statistics. The data sets look like they're pretty clean in its current state, but grouping by state and aggragating to generate summary data will get rid of any missing or null values.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
A Redshift database was chosen for easy access to perform time series analysis. The final table schema will have the aggregated daily data (containing state, day (YYYY-MM-DD), count, min, max, mean, std) and a `JOIN`-ed table containing the daily maximums and minimums by state and day. 

#### 3.2 Mapping Out Data Pipelines
+ The data will be transformed locally using Pandas and saved to new CSV files.
    + loading the data from zip files ended up being the slowest part because the zip files are extremely compressed and end up 2-3Gb.
+ The new CSV files will be uploaded to S3 for faster loading into Redshift.
    + saving to a new file and uploading the files and using `COPY` turned out to be faster than bulk `INSERT`.
+ New mean and maximum tables will be created by joining all means and maximums from the daily data tables.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

##### Transform the Data to Daily Aggregations by State

In [ ]:
# transform the data in CSV files to aggregate values:
#   state, day (YYYY-MM-DD), count, min, max, mean, std
# then save to a new CSV file and upload to S3
#
# years imported above
# years is a `range` of years
# years = range(start, end+1)
#
# code_dict imported above
# code_dict is a dictionary corresponding to names and EPA codes
# example:
#   code_dict = {'Temperature (62101)': 'TEMP',
#                 'Ozone': 44201,
#                 'SO2': 42401,
#                 'CO': 42101,
#                 'NO2': 42602}

# get AWS credentials from config
config = configparser.ConfigParser(allow_no_value=True)
config.read('dwh.cfg')
# create s3 session using AWS credentials
session = boto3.Session(aws_access_key_id = config.get('AWS','KEY'),
                    aws_secret_access_key= config.get('AWS','SECRET'))
s3 = session.resource('s3')
# get s3 bucket name to save files
s3_bucket_name = config.get('S3','bucket_name')

# get data file names and iterate through them
for title, code in code_dict.items():
    working_dir = os.path.join(data_folder_name, title)
    files = map(lambda file_name: os.path.join(working_dir, file_name),
                os.listdir(working_dir))
    for file in files:
        print(f'loading {file}')
        df = pd.read_csv(file, dtype={'Qualifier': 'object',
                                      'Date Local': 'object',
                                      'Time Local': 'object',
                                      'Date GMT': 'object',
                                      'Time GMT': 'object'})
        print(f'transforming {file}')
        # drop all columns but 'State Name', 'Date Local', 'Sample Measurement'
        df = df[['State Name', 'Date Local', 'Sample Measurement']]
        # rename columns to one word for easier reference
        df.rename(columns={"State Name": "state",
                           "Date Local": "date",
                           "Sample Measurement": title},
                  inplace=True)
        # group by and aggregate
        # note: the aggregate function `agg` ended up being much faster than the `.describe()` method.
        df = df.groupby(['state', 'date'], as_index=False)[title] \
               .agg([pd.Series.count, np.min, np.max, np.mean, np.std]) \
               .reset_index() \
               .rename(columns={"amin": "min",
                                "amax": "max"}) \
               .astype({"count": "int64"})
        # save transformed data and upload to S3
        year = file.split('.')[-2][-4:]
        transformed_file = f'daily_{title}_{year}.csv'
        save_path = os.path.join(working_dir, transformed_file)
        print(f'saving transformed data to {save_path}')
        df.to_csv(save_path, index=False)
        print(f'uploading {save_path} to S3')
        s3.Bucket(s3_bucket_name).upload_file(save_path, transformed_file)
        # clean up local data files
        print(f'deleting {file}')
        os.remove(file)
        print(f'deleting {save_path}')
        os.remove(save_path)
        print('')
    
# delete data folder   
os.system(f'rm -r {data_folder_name}')

##### Create Database Connection

In [ ]:
config = configparser.ConfigParser(allow_no_value=True)
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

##### Drop Tables (If They Exist)

In [ ]:
drop_tables = list(map(lambda x: x.lower(), code_dict.keys())) \
            + ['measure_mean', 'measure_max']
for table in drop_tables:
    print(f'DROP TABLE IF EXISTS {table}')
    cur.execute(f'DROP TABLE IF EXISTS {table}')
    conn.commit()

##### Create Tables

In [ ]:
epa_codes = list(map(lambda x: x.lower(), code_dict.keys()))
for table in epa_codes:
    create_table_query = \
        f"""CREATE TABLE IF NOT EXISTS {table}
            (state TEXT,
             date DATE,
             count INT,
             min DECIMAL,
             max DECIMAL,
             mean DECIMAL,
             std DECIMAL,
             PRIMARY KEY (state, date)
            );
        """
    print(create_table_query)
    cur.execute(create_table_query)
    conn.commit()

# build and commit queries for mean and max tables
create_pivot_tables = ['measure_mean', 'measure_max']
for table in create_pivot_tables:
    # build query
    create_pivot_table_query = \
        f"""CREATE TABLE IF NOT EXISTS {table}
            (state TEXT, date DATE, """ \
            + "".join(map(lambda x: x + " DECIMAL, ", epa_codes)) \
            + "PRIMARY KEY (state, date));"
    # create table from query
    print(create_pivot_table_query)
    cur.execute(create_pivot_table_query)
    conn.commit()

##### Load Tables

In [ ]:
access_key = config.get('AWS','KEY')
secret_key = config.get('AWS','SECRET')

In [ ]:
# load daily tables
for table in code_dict.keys():
    daily_table_copy = (f"""
        COPY {table.lower()}
        FROM 's3://{s3_bucket_name}/daily_{table}'
        access_key_id '{access_key}'
        secret_access_key '{secret_key}'
        FORMAT AS CSV
        IGNOREHEADER 1;
        """)
    print(daily_table_copy)
    cur.execute(daily_table_copy)
    conn.commit()

In [ ]:
# load JOIN-ed tables
tables = list(map(lambda x: x.lower(), code_dict.keys()))
for agg_table in ['measure_mean', 'measure_max']:
    query = f'INSERT INTO {agg_table} (state, "date", ' + ", ".join(tables) + ") " + \
            f"SELECT {tables[0]}.state, {tables[0]}.date, " + \
            ", ".join(map(lambda x: x + ".{} as {}".format(agg_table.split('_')[-1], x), tables)) + \
            f" FROM {tables[0]} " + \
            " ".join(f"JOIN {table} on ({table}.state = {tables[0]}.state and {table}.date = {tables[0]}.date)" for table in tables[1:]) 
    print(query)
    cur.execute(query)
    conn.commit()

##### Close Connection

In [ ]:
conn.close()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

##### Create Database Connection

In [ ]:
config = configparser.ConfigParser(allow_no_value=True)
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

##### Check Table Schema

In [ ]:
schema = sqlio.read_sql_query("""SELECT * FROM PG_TABLE_DEF""", conn)
schema = schema[schema.schemaname == 'public'][['tablename', 'column', 'type']]

In [ ]:
# use .head() to dipslay all of the table schema
schema.groupby('tablename', as_index=False).head(max(2 + len(code_dict.keys()), 7))

##### Check that Tables Loaded Into Redshift

In [ ]:
sqlio.read_sql_query("""SELECT "table", tbl_rows FROM SVV_TABLE_INFO""", conn).astype({"tbl_rows": "int64"})

##### Close Connection

In [ ]:
conn.close()

#### 4.3 Data dictionary 
The data will ultimately consist of tables of summary statistics by day and state. The summary statistics will be used to create tables of daily maximums and averages for all the tables. The final schema will depend on the what is indicated in `code_dict`, but a generic table schema can be seen below with the column types.

<img src="images/schema.png" alt="schema" style="width:80%">

#### Step 5: Complete Project Write Up
Some of the rational of tools has been discussed throughout, but will be reviewed here. The end-product of a Redshift database was chosen due to scalability of size and access as well as ease of management as an AWS cloud product. Due to the data set chunk size of the zipped CSVs, the data set files can easily be unzipped and transformed using Pandas. Uploading the transformed data into S3 to use Redshift's `COPY` ability was chosen because it was ultimately faster than bulk `INSERT`-ing directly into the Redshift database using Pandas `to_sql` functionality or using the psycopg2 library directly. COPY is likely faster due to the parallelization and upload of transformed data is very fast due to the small file size.

The code in the notebook is flexible based on the `code_dict`, to download files for different gases/particulates and years. The notebook scans the directories, dynamically generates SQL code based on the `code_dict` and `years`, and Redshift uses partial, file name matching, so the amount of data can vary but the end result will be to have a daily summary table for each thing selected in `code_dict` and summary tables with daily averages and maximums.

##### Addressing Pipeline Concerns
If incremental updates need to be added (like daily updates), then there are several options:
 + [Air Quality System (AQS) API](https://aqs.epa.gov/aqsweb/documents/data_api.html): API access to data that will allow retreival of newest data.
 + [AirNow API](https://docs.airnowapi.org): Another provider that supplies real-time air quality data.
 + Use an Apache Airflow pipeline or a `cron` job with a Python script to redownload the zip file daily and only UPSERT the newest data.

If the datasets required are increased 100x, then the pipeline needs to be changed. The current VM has no problem loading one dataset at a time for transformation, but directly loading the zip files into Redshift using `COPY` may be an option depending on the zip compression (only certain ones are supported). Alternatively, the AQS or AirNow API could also be used to obtain the data.

If the dataset needs to be accessed by 100+ people, Redshift would have no problem scaling. It would require more money in user access, but all the heavy lifting will be done by AWS requiring only to grant user permissions with additional IAM roles.